# _Imports and Opening Datasets_


In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [91]:
animes_path = "../data/AnimeList.csv"
users_path = "../data/UserList.csv"
reviews_path = "../data/UserAnimeList.parquet"


In [92]:
animes_df = pd.read_csv(animes_path)
users_df = pd.read_csv(users_path)
reviews_df = pd.read_parquet(reviews_path, columns=["username", "anime_id", "my_score"])


In [94]:
animes_df["title_english"]
anime_df[anime_df["title_english"] == "Bakuman."]

0        Inu X Boku Secret Service
1            My Bride is a Mermaid
2               Shugo Chara!! Doki
3                    Princess Tutu
4                         Bakuman.
                   ...            
14473                          NaN
14474                          NaN
14475                          NaN
14476                          NaN
14477                          NaN
Name: title_english, Length: 14478, dtype: object

# Reviews_df Exploration & Preparation


In [82]:
reviews_df.head()


,username,anime_id,my_score
13587253,Golden_Truth,5460,6
44469336,Choco22,33338,9
62177081,Daixu-De-Castro,20767,8
19970912,ONDORE,1689,0
54515097,-Nel-,2273,0


In [83]:
all_df = pd.merge(
    reviews_df, animes_df[["anime_id", "title"]], how="left", on="anime_id"
)
all_df.head()


,username,anime_id,my_score,title
0,Golden_Truth,5460,6,Detective Conan Movie 13: The Raven Chaser
1,Choco22,33338,9,One Piece: Heart of Gold
2,Daixu-De-Castro,20767,8,Noragami OVA
3,ONDORE,1689,0,Byousoku 5 Centimeter
4,-Nel-,2273,0,Mobile Suit Gundam Wing: Endless Waltz Movie


In [84]:
count_series = all_df["title"].value_counts()

all_df = all_df[all_df.title.isin(count_series[count_series > 2000].index)]
all_df.head()


,username,anime_id,my_score,title
2,Daixu-De-Castro,20767,8,Noragami OVA
3,ONDORE,1689,0,Byousoku 5 Centimeter
5,Anastasiaromanov,7674,0,Bakuman.
6,mdlnbrns,28735,0,Shouwa Genroku Rakugo Shinjuu
7,i_want_ultima,11759,2,Accel World


In [85]:
reviews_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)
all_df.drop_duplicates(subset=["username", "anime_id"], keep="last", inplace=True)

score_matrix_df = all_df.pivot_table(
    index="username", columns="title", values="my_score"
)


In [ ]:
# score_matrix_df.to_parquet('../data/score_matrix.parquet')

In [86]:
def recommendation_system(anime_name):

    # grab user ratings for the a certain anime
    anime_user_ratings = score_matrix_df[anime_name]

    # Use Corrwith as a method to get user correlation
    similar_to_anime = score_matrix_df.corrwith(anime_user_ratings)

    # Clean the null values from both movies
    corr_anime = pd.DataFrame(similar_to_anime, columns=["Correlation"])

    # sort dataframe by correlation and choosing only the top 10 anime
    return corr_anime.sort_values("Correlation", ascending=False).head(10).reset_index()


In [87]:
recommendation_system("Bleach")


,title,Correlation
0,Bleach,1.000000
1,Bleach: The Sealed Sword Frenzy,0.573518
2,Fairy Tail OVA,0.563693
3,Bleach Movie 4: Jigoku-hen,0.561095
4,Bleach Movie 3: Fade to Black - Kimi no Na wo ...,0.529737
5,Bleach Movie 1: Memories of Nobody,0.520337
6,Bleach: Memories in the Rain,0.511359
7,Naruto: Shippuuden,0.510672
8,Dragon Ball Z Movie 12: Fukkatsu no Fusion!! G...,0.487466
9,Naruto,0.475261


In [88]:
recommendation_system("Dragon Ball")


,title,Correlation
0,Dragon Ball,1.000000
1,Dragon Ball Z,0.825568
2,Dragon Ball Z Special 2: Zetsubou e no Hankou!...,0.706778
3,Dragon Ball GT,0.676084
4,Dragon Ball Z Movie 02: Kono Yo de Ichiban Tsu...,0.648749
5,Digimon Adventure 02,0.612159
6,Pokemon: Mewtwo! Ware wa Koko ni Ari,0.606598
7,Dragon Ball Z Movie 01: Ora no Gohan wo Kaese!!,0.603859
8,Dragon Ball Z Movie 04: Super Saiyajin da Son ...,0.602274
9,Dragon Ball Z Special 1: Tatta Hitori no Saish...,0.596030


In [89]:
recommendation_system("One Outs")


,title,Correlation
0,One Outs,1.000000
1,Astarotte no Omocha!,0.666854
2,Maburaho,0.625263
3,Trigun: Badlands Rumble,0.613353
4,Shinryaku!? Ika Musume,0.604001
5,Afro Samurai: Resurrection,0.596304
6,Kimi ga Aruji de Shitsuji ga Ore de,0.589950
7,Samurai 7,0.587110
8,Hoshizora e Kakaru Hashi,0.585131
9,Zan Sayonara Zetsubou Sensei,0.582201
